# **Homework 1: COVID-19 Cases Prediction (Regression)**

Objectives:
* Solve a regression problem with deep neural networks (DNN).
* Understand basic DNN training tips.
* Familiarize yourself with PyTorch.

If you have any questions, please contact the TAs via TA hours, NTU COOL, or email to mlta-2023-spring@googlegroups.com

In [1]:
# check gpu type
!nvidia-smi

Sat Mar  4 21:35:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
| 53%   46C    P2   122W / 350W |   1055MiB / 24268MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Download data
If the Google Drive links below do not work, you can use the dropbox link below or download data from [Kaggle](https://www.kaggle.com/competitions/ml2023spring-hw1/overview), and upload data manually to the workspace.

In [2]:
# google drive link
# !gdown --id '1BjXalPZxq9mybPKNjF3h5L3NcF7XKTS-' --output covid_train.csv
# !gdown --id '1B55t74Jg2E5FCsKCsUEkPKIuqaY7UIi1' --output covid_test.csv

# dropbox link
!wget -O covid_train.csv https://www.dropbox.com/s/lmy1riadzoy0ahw/covid.train.csv?dl=0
!wget -O covid_test.csv https://www.dropbox.com/s/zalbw42lu4nmhr2/covid.test.csv?dl=0

--2023-03-04 21:35:23--  https://www.dropbox.com/s/lmy1riadzoy0ahw/covid.train.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.85.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.85.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/lmy1riadzoy0ahw/covid.train.csv [following]
--2023-03-04 21:35:23--  https://www.dropbox.com/s/raw/lmy1riadzoy0ahw/covid.train.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc240c809eda59050830a2bafd33.dl.dropboxusercontent.com/cd/0/inline/B3mci3bxaaAUOyie66FWBjVptNnaLEmk40lvf_ApbFPZdt59MffnRYInQwzKJL1w08yvirviq6cGoCg83jMsaz8fokZ5agG5w5jvIV1dCl9YbMv3P43Sf2fHRif38OmqGYKFCEQyb7PD3pfdfCdegBtG0wWCaJMsNaQR1J4BrVNuew/file# [following]
--2023-03-04 21:35:24--  https://uc240c809eda59050830a2bafd33.dl.dropboxusercontent.com/cd/0/inline/B3mci3bxaaAUOyie66FWBjVptNnaLEmk40lvf_ApbFPZdt59MffnRYInQwzKJL1

# Import packages

In [3]:
# Numerical Operations
import math
import numpy as np

# Reading/Writing Data
import pandas as pd
import os
import csv

# For Progress Bar
from tqdm import tqdm

# Pytorch
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter

/home/tonic/.conda/envs/py37DRL/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Some Utility Functions

You do not need to modify this part.

In [4]:
def same_seed(seed): 
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def train_valid_split(data_set, valid_ratio, seed):
    '''Split provided training data into training set and validation set'''
    valid_set_size = int(valid_ratio * len(data_set)) 
    train_set_size = len(data_set) - valid_set_size
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    return np.array(train_set), np.array(valid_set)

def predict(test_loader, model, device):
    model.eval() # Set your model to evaluation mode.
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)                        
        with torch.no_grad():                   
            pred = model(x)                     
            preds.append(pred.detach().cpu())   
    preds = torch.cat(preds, dim=0).numpy()  
    return preds

# Dataset

In [5]:
class COVID19Dataset(Dataset):
    '''
    x: Features.
    y: Targets, if none, do prediction.
    '''
    def __init__(self, x, y=None):
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)

    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)

# Neural Network Model
Try out different model architectures by modifying the class below.

In [6]:
class My_Model(nn.Module):
    def __init__(self, input_dim):
        super(My_Model, self).__init__()
        # TODO: modify model's structure, be aware of dimensions. 
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1) # (B, 1) -> (B)
        return x

# Feature Selection
Choose features you deem useful by modifying the function below.

In [7]:
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import SelectKBest, chi2,f_regression



def select_feat(train_data, valid_data, test_data, select_all=False,kk=40):
    '''Selects useful features to perform regression'''
    y_train, y_valid = train_data[:,-1], valid_data[:,-1]
    raw_x_train, raw_x_valid, raw_x_test = train_data[:,:-1], valid_data[:,:-1], test_data
    
    if select_all:
        feat_idx = list(range(raw_x_train.shape[1]))
    else:
        feat_idx=[]
        
        model = SelectKBest(f_regression, k=kk)
        #print(raw_x_valid.shape, y_valid.shape)
        #print("score:",model.scores_)
        model.fit(raw_x_train,y_train)
        #print("score:",model.scores_)
        feat_idx=model.get_support(indices=True)
        
        '''
        for i in range(35,raw_x_train.shape[1]):
          if i in choose_list:
            feat_idx.append(i)
        '''
        
        
        
        
    print(feat_idx)
    features=pd.read_csv('./covid_train.csv').columns
    print(features[feat_idx])
        
    return raw_x_train[:,feat_idx], raw_x_valid[:,feat_idx], raw_x_test[:,feat_idx], y_train, y_valid

# Training Loop

In [8]:
def trainer(train_loader, valid_loader, model, config, device,k,currentDateAndTime):

    criterion = nn.MSELoss(reduction='mean') # Define your loss function, do not modify this.

    # Define your optimization algorithm. 
    # TODO: Please check https://pytorch.org/docs/stable/optim.html to get more available algorithms.
    # TODO: L2 regularization (optimizer(weight decay...) or implement by your self).
    optimizer=0
    if config['optimizer']=="Adam":
        optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'], weight_decay=config['weightDecay'],amsgrad=config['amsgrad'],eps=config['eps'],betas=config['betas'])
    elif config['optimizer']=="RMSprop":
        optimizer = torch.optim.RMSprop(model.parameters(), lr=config['learning_rate'], momentum=0.8, weight_decay=config['weightDecay']) 
    elif config['optimizer']=="Adagrad":
        optimizer = torch.optim.Adagrad(model.parameters(), lr=config['learning_rate'], weight_decay=config['weightDecay'])  
    elif config['optimizer']=="AdamW":
        optimizer = torch.optim.AdamW(model.parameters(), lr=config['learning_rate'], weight_decay=config['weightDecay'],amsgrad=config['amsgrad'],eps=config['eps'],betas=config['betas'])
    scheduler=torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max=20)
    
    

    writer = SummaryWriter() # Writer of tensoboard.

    if not os.path.isdir('./models'):
        os.mkdir('./models') # Create directory of saving models.

    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

    for epoch in range(n_epochs):
        model.train() # Set your model to train mode.
        loss_record = []

        # tqdm is a package to visualize your training progress.
        #train_pbar = tqdm(train_loader, position=0, leave=True)

        for x, y in train_loader:
            optimizer.zero_grad()               # Set gradient to zero.
            x, y = x.to(device), y.to(device)   # Move your data to device. 
            pred = model(x)             
            loss = criterion(pred, y)
            loss.backward()                     # Compute gradient(backpropagation).
            optimizer.step()                    # Update parameters.
            step += 1
            loss_record.append(loss.detach().item())
        scheduler.step()
            
            # Display current epoch number and loss on tqdm progress bar.
            #train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
            #train_pbar.set_postfix({'loss': loss.detach().item()})

        mean_train_loss = sum(loss_record)/len(loss_record)
        writer.add_scalar('Loss/train', mean_train_loss, step)

        model.eval() # Set your model to evaluation mode.
        loss_record = []
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)

            loss_record.append(loss.item())
            
        mean_valid_loss = sum(loss_record)/len(loss_record)
        sourceFile = open('demo/demo_'+str(currentDateAndTime)+'.txt', 'a+')
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}', file = sourceFile)
        sourceFile.close()
        # writer.add_scalar('Loss/valid', mean_valid_loss, step)

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path']) # Save your best model
            

            #print('Saving model with loss {:.3f}...'.format(best_loss))
            
            early_stop_count = 0
        else: 
            early_stop_count += 1

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            sourceFile = open('demo/demo_'+str(currentDateAndTime)+'.txt', 'a+')
            print(best_loss, file = sourceFile)
            sourceFile.close()
            return best_loss
    sourceFile = open('demo/demo_'+str(currentDateAndTime)+'.txt', 'a+')
    print(best_loss, file = sourceFile)
    sourceFile.close()
    return best_loss

# Configurations
`config` contains hyper-parameters for training and the path to save your model.

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = {
    'seed': 5201314 ,    # Your seed number, you can pick your lucky number. :)
    'select_all': False,   # Whether to use all features.
    'valid_ratio': 0.2,   # validation_size = train_size * valid_ratio
    'n_epochs': 10000,     # Number of epochs.            
    'batch_size': 256, 
    'learning_rate': 2e-3,              
    'early_stop': 600,    # If model has not improved for this many consecutive epochs, stop training.     
    'save_path': './models/model.ckpt' , # Your model will be saved herE
    'weightDecay': 5e-4,
    'optimizer': 'AdamW',
    'amsgrad':True,
    'eps':1e-8,
    'betas':(0.9, 0.999)
}


# Dataloader
Read data from files and set up training, validation, and testing sets. You do not need to modify this part.

In [10]:
#from sklearn.preprocessing import MinMaxScaler
def get_data_loader(k):
  same_seed(config['seed'])
  train_data, test_data = pd.read_csv('./covid_train.csv').values, pd.read_csv('./covid_test.csv').values
  train_data, valid_data = train_valid_split(train_data, config['valid_ratio'], config['seed'])
  # Select features
  x_train, x_valid, x_test, y_train, y_valid = select_feat(train_data, valid_data, test_data,config['select_all'],k)
  #x_train, x_valid, x_test, y_train, y_valid=pca(x_train, x_valid, x_test, y_train, y_valid)
  # Print out the data size.
  '''
  scaler = MinMaxScaler()
  x_train = scaler.fit_transform(x_train)
  x_valid = scaler.fit_transform(x_valid)
  x_test = scaler.transform(x_test)
  '''
  #print(x_train)
  print(f"""train_data size: {train_data.shape} 
  valid_data size: {valid_data.shape} 
  test_data size: {test_data.shape}""")



  # Print out the number of features.
  print(f'number of features: {x_train.shape[1]}')

  train_dataset, valid_dataset, test_dataset = COVID19Dataset(x_train, y_train), \
                                              COVID19Dataset(x_valid, y_valid), \
                                              COVID19Dataset(x_test)

  # Pytorch data loader loads pytorch dataset into batches.
  train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
  valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
  test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)
  return train_loader,valid_loader,test_loader,x_train

# Start training!

### Method1

In [11]:
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVC
def getLinearRegressionResult():
    train_data, test_data = pd.read_csv('./covid_train.csv').values, pd.read_csv('./covid_test.csv').values
    train_data, valid_data = train_valid_split(train_data, 0.2, config['seed'])
    x_train, x_valid, x_test, y_train, y_valid = select_feat(train_data, valid_data, test_data,config['select_all'],k)

    #model = XGBRegressor(n_estimators=10000, max_depth=1000, eta=0.1, subsample=0.7, colsample_bytree=0.8)
    #model=  XGBRegressor()
    model =LinearRegression()

    model.fit(x_train, y_train)
    # define model evaluation method
    #print(model.predict(x_test))
    pred=model.predict(x_valid)
    #print(model.score(x_train, y_train))
    err=mean_squared_error(pred,y_valid)
    #print(err)
    pred=model.predict(x_test)
    return pred
#pred=getLinearRegressionResult()

In [12]:
def save_pred(preds, file):
  ''' Save predictions to specified file '''
  with open(file, 'w') as fp:
      writer = csv.writer(fp)
      writer.writerow(['id', 'tested_positive'])
      for i, p in enumerate(preds):
          writer.writerow([i, p])

In [13]:
from datetime import datetime
try:
    os.mkdir("demo")
except:
    pass
currentDateAndTime = datetime.now()
k=17
train_loader,valid_loader,test_loader,x_train=get_data_loader(k)
sourceFile = open('demo/demo_'+str(currentDateAndTime)+'.txt', 'w')

model = My_Model(input_dim=x_train.shape[1]).to(device) # put your model and data on the same computation device.
print(config, model,file = sourceFile)

sourceFile.close()
loss=trainer(train_loader, valid_loader, model, config, device,k,currentDateAndTime)

print(loss)
print(config)
print(model)

[35 36 37 47 48 52 53 54 55 65 66 70 71 72 73 83 84]
Index(['cli', 'ili', 'wnohh_cmnty_cli', 'hh_cmnty_cli', 'nohh_cmnty_cli',
       'tested_positive', 'cli.1', 'ili.1', 'wnohh_cmnty_cli.1',
       'hh_cmnty_cli.1', 'nohh_cmnty_cli.1', 'tested_positive.1', 'cli.2',
       'ili.2', 'wnohh_cmnty_cli.2', 'hh_cmnty_cli.2', 'nohh_cmnty_cli.2'],
      dtype='object')
train_data size: (2408, 89) 
  valid_data size: (601, 89) 
  test_data size: (997, 88)
number of features: 17

Model is not improving, so we halt the training session.
0.747297336657842
{'seed': 5201314, 'select_all': False, 'valid_ratio': 0.2, 'n_epochs': 10000, 'batch_size': 256, 'learning_rate': 0.002, 'early_stop': 600, 'save_path': './models/model.ckpt', 'weightDecay': 0.0005, 'optimizer': 'AdamW', 'amsgrad': True, 'eps': 1e-08, 'betas': (0.9, 0.999)}
My_Model(
  (layers): Sequential(
    (0): Linear(in_features=17, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=16, bias=True)
    

### Method2

In [14]:
from datetime import datetime
currentDateAndTime = datetime.now()
k=17
train_loader,valid_loader,test_loader,x_train=get_data_loader(k)
sourceFile = open('demo/demo_'+str(currentDateAndTime)+'.txt', 'w')

model = My_Model(input_dim=x_train.shape[1]).to(device) # put your model and data on the same computation device.
print(config, model,file = sourceFile)

sourceFile.close()
loss=trainer(train_loader, valid_loader, model, config, device,k,currentDateAndTime)

print(loss)
print(config)
print(model)

[35 36 37 47 48 52 53 54 55 65 66 70 71 72 73 83 84]
Index(['cli', 'ili', 'wnohh_cmnty_cli', 'hh_cmnty_cli', 'nohh_cmnty_cli',
       'tested_positive', 'cli.1', 'ili.1', 'wnohh_cmnty_cli.1',
       'hh_cmnty_cli.1', 'nohh_cmnty_cli.1', 'tested_positive.1', 'cli.2',
       'ili.2', 'wnohh_cmnty_cli.2', 'hh_cmnty_cli.2', 'nohh_cmnty_cli.2'],
      dtype='object')
train_data size: (2408, 89) 
  valid_data size: (601, 89) 
  test_data size: (997, 88)
number of features: 17

Model is not improving, so we halt the training session.
0.747297336657842
{'seed': 5201314, 'select_all': False, 'valid_ratio': 0.2, 'n_epochs': 10000, 'batch_size': 256, 'learning_rate': 0.002, 'early_stop': 600, 'save_path': './models/model.ckpt', 'weightDecay': 0.0005, 'optimizer': 'AdamW', 'amsgrad': True, 'eps': 1e-08, 'betas': (0.9, 0.999)}
My_Model(
  (layers): Sequential(
    (0): Linear(in_features=17, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=16, bias=True)
    

In [15]:
config = {
    'seed': 5201314 ,    # Your seed number, you can pick your lucky number. :)
    'select_all': False,   # Whether to use all features.
    'valid_ratio': 0.2,   # validation_size = train_size * valid_ratio
    'n_epochs': 10000,     # Number of epochs.            
    'batch_size': 256, 
    'learning_rate': 2e-3,              
    'early_stop': 600,    # If model has not improved for this many consecutive epochs, stop training.     
    'save_path': './models/model2.ckpt' , # Your model will be saved herE
    'weightDecay': 5e-4,
    'optimizer': 'RMSprop',
    'amsgrad':True,
    'eps':1e-8,
    'betas':(0.9, 0.999)
}

In [16]:
from datetime import datetime
currentDateAndTime = datetime.now()
k=17
train_loader,valid_loader,test_loader,x_train=get_data_loader(k)
sourceFile = open('demo/demo_'+str(currentDateAndTime)+'.txt', 'w')

model2 = My_Model(input_dim=x_train.shape[1]).to(device) # put your model and data on the same computation device.
print(config, model,file = sourceFile)

sourceFile.close()
loss=trainer(train_loader, valid_loader, model, config, device,k,currentDateAndTime)

print(loss)
print(config)
print(model)

[35 36 37 47 48 52 53 54 55 65 66 70 71 72 73 83 84]
Index(['cli', 'ili', 'wnohh_cmnty_cli', 'hh_cmnty_cli', 'nohh_cmnty_cli',
       'tested_positive', 'cli.1', 'ili.1', 'wnohh_cmnty_cli.1',
       'hh_cmnty_cli.1', 'nohh_cmnty_cli.1', 'tested_positive.1', 'cli.2',
       'ili.2', 'wnohh_cmnty_cli.2', 'hh_cmnty_cli.2', 'nohh_cmnty_cli.2'],
      dtype='object')
train_data size: (2408, 89) 
  valid_data size: (601, 89) 
  test_data size: (997, 88)
number of features: 17

Model is not improving, so we halt the training session.
0.7632689674695333
{'seed': 5201314, 'select_all': False, 'valid_ratio': 0.2, 'n_epochs': 10000, 'batch_size': 256, 'learning_rate': 0.002, 'early_stop': 600, 'save_path': './models/model2.ckpt', 'weightDecay': 0.0005, 'optimizer': 'RMSprop', 'amsgrad': True, 'eps': 1e-08, 'betas': (0.9, 0.999)}
My_Model(
  (layers): Sequential(
    (0): Linear(in_features=17, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=16, bias=True)


### Testing
The predictions of your model on testing set will be stored at `pred.csv`.

In [17]:

def save_pred(preds, file):
    ''' Save predictions to specified file '''
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'tested_positive'])
        for i, p in enumerate(preds):
            writer.writerow([i, p])

model = My_Model(input_dim=x_train.shape[1]).to(device)
model.load_state_dict(torch.load('./models/model.ckpt'))
model2 = My_Model(input_dim=x_train.shape[1]).to(device)
model2.load_state_dict(torch.load('./models/model2.ckpt'))
preds = predict(test_loader, model, device)
preds2= predict(test_loader, model2, device)
preds=(preds+preds2)/2
save_pred(preds, 'pred2.csv')

100%|██████████| 4/4 [00:00<00:00, 1597.37it/s]


# Plot learning curves with `tensorboard` (optional)

`tensorboard` is a tool that allows you to visualize your training progress.

If this block does not display your learning curve, please wait for few minutes, and re-run this block. It might take some time to load your logging information. 

In [18]:
%reload_ext tensorboard
%tensorboard --logdir=./runs/

Reusing TensorBoard on port 6006 (pid 917056), started 12:44:00 ago. (Use '!kill 917056' to kill it.)

# Download

Run this block to download the `pred.csv` automatically.

In [19]:
from google.colab import files
files.download('pred.csv')

ModuleNotFoundError: No module named 'google.colab'

# Reference
This notebook uses code written by Heng-Jui Chang @ NTUEE (https://github.com/ga642381/ML2021-Spring/blob/main/HW01/HW01.ipynb)

: 